In [251]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import math
import pyodbc
from config import av_api_key as api_key, seeking_alpha_name, seeking_alpha_pw

# Import dependencies for web scraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import numpy as np
import os

# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [252]:
def load_access_table_to_df(access_file_path, table_name):
    """
    Loads a table from a Microsoft Access database into a Pandas DataFrame.

    Parameters:
    access_file_path (str): The full path to the Microsoft Access database file (.mdb or .accdb).
    table_name (str): The name of the table to load from the database.

    Returns:
    pd.DataFrame: The DataFrame containing the data from the specified table.

    Notes:
    - Requires the pyodbc library and the Microsoft Access Driver installed on your system.
    - Ensure the bit version (32-bit or 64-bit) of Python matches the installed Access Driver.
    """
    # Connection string for Microsoft Access
    conn_str = (
        r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"
        r"DBQ=" + access_file_path + ";"
    )
    
    # Establish connection
    conn = pyodbc.connect(conn_str)
    
    # SQL query to select all data from the table
    query = f"SELECT * FROM {table_name}"
    
    # Read data into DataFrame
    df = pd.read_sql(query, conn)
    
    # Close the connection
    conn.close()
    
    return df

def create_db_dfs(table_name='brokerage'):
    # Path to database
    access_file_path = 'c:/users/culle/onedrive/documents/finances/investments.accdb'

    # Individual tables in database
    table_name = table_name
    
    return load_access_table_to_df(access_file_path,table_name)

In [253]:
def create_summary_dataframe_with_weighted_date(df, reference_date=None):
    """
    Creates a summary DataFrame by aggregating transaction data for each company (TICKER),
    including a weighted average purchase date based on total cost (quantity * purchase price).

    The summary includes:
    - TICKER: The company ticker.
    - ASSET_CLASS: The asset class (assumed consistent per TICKER, takes the first occurrence).
    - SECTOR: The sector (assumed consistent per TICKER, takes the first occurrence).
    - WEIGHTED_AVG_PURCHASE_DATE: The cost-weighted average purchase date.
    - TOTAL_COST: The total cost of all purchases.
    - TOTAL_QUANTITY: The total quantity of shares purchased.
    - AVG_PURCHASE_PRICE: The weighted average purchase price (total cost / total quantity).

    Parameters:
    df (pd.DataFrame): The input DataFrame with columns: ID, TICKER, ASSET_CLASS, SECTOR,
                      ACQUIRED, PURCHASE_PRICE, QUANTITY.
    reference_date (str): Reference date for date-to-numeric conversion (default: '2025-01-01').

    Returns:
    pd.DataFrame: The summary DataFrame with aggregated data per TICKER, including weighted purchase date.
    """
    # Set reference_date to current date if None
    if reference_date is None:
        reference_date = datetime.now().date()

    # Ensure ACQUIRED is in datetime format
    df['ACQUIRED'] = pd.to_datetime(df['ACQUIRED'])
    
    # Calculate the cost for each transaction
    df['COST'] = df['PURCHASE_PRICE'] * df['QUANTITY']
    
    # Convert ACQUIRED date to numeric (days since reference_date)
    reference_date = pd.to_datetime(reference_date)
    df['DAYS_SINCE_REF'] = (df['ACQUIRED'] - reference_date).dt.days
    
    # Calculate weighted days (cost * days since reference)
    df['WEIGHTED_DAYS'] = df['COST'] * df['DAYS_SINCE_REF']
    
    # Group by TICKER and aggregate
    summary_df = df.groupby('TICKER').agg({
        'ASSET_CLASS': 'first',
        'ALPHA_PICKED': 'first',
        'QUANT_RATING': 'first',
        'SECTOR': 'first',
        'ACQUIRED': 'min',
        'QUANTITY': 'sum',
        'COST': 'sum',
        'WEIGHTED_DAYS': 'sum'
    }).reset_index()
    
    # Rename columns for clarity
    summary_df = summary_df.rename(columns={
        'ACQUIRED': 'FIRST_ACQUIRED',
        'QUANTITY': 'TOTAL_QUANTITY',
        'COST': 'TOTAL_COST'
    })
    
    # Calculate weighted average purchase price
    summary_df['AVG_PURCHASE_PRICE'] = round(summary_df['TOTAL_COST'] / summary_df['TOTAL_QUANTITY'], ndigits=3)
    
    # Calculate weighted average days
    summary_df['WEIGHTED_AVG_DAYS'] = summary_df['WEIGHTED_DAYS'] / summary_df['TOTAL_COST']
    
    # Convert weighted average days back to a datetime
    summary_df['WEIGHTED_AVG_PURCHASE_DATE'] = reference_date + pd.to_timedelta(summary_df['WEIGHTED_AVG_DAYS'], unit='D')
    
    # Round the weighted average date to the nearest day for clarity
    summary_df['WEIGHTED_AVG_PURCHASE_DATE'] = summary_df['WEIGHTED_AVG_PURCHASE_DATE'].dt.round('D')
    
    # Reorder columns as requested
    columns_order = [
        'TICKER', 'ALPHA_PICKED', 'QUANT_RATING', 'ASSET_CLASS', 'SECTOR', 'WEIGHTED_AVG_PURCHASE_DATE',
        'TOTAL_COST', 'TOTAL_QUANTITY', 'AVG_PURCHASE_PRICE'
    ]
    summary_df = summary_df[columns_order]
    
    # Drop temporary columns from input DataFrame
    df.drop(['COST', 'DAYS_SINCE_REF', 'WEIGHTED_DAYS'], axis=1, inplace=True)
    
    return summary_df

In [254]:
# Load database tables
r_analysis = create_db_dfs('retirement')
alpha_raw = create_db_dfs('alpha_picks')
watch_list = create_db_dfs('watch_list')
b_analysis = create_db_dfs('brokerage')

C:\Users\culle\AppData\Local\Temp\ipykernel_38204\1991132214.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [255]:
# Merge portfolios and watch lists to create master portfolio
r_analysis2 = pd.merge(
        r_analysis,
        alpha_raw,  # Drop BASIS as it's redundant with AVG_PURCHASE_PRICE
        on='TICKER',
        how='outer'
    )
r_analysis2.drop(columns=['ID_x','ID_y'], inplace=True)

alpha_plus_watch = pd.concat([alpha_raw, watch_list], ignore_index=True)

b_analysis2 = pd.merge(
    b_analysis,
    alpha_plus_watch,
    on='TICKER',
    how='outer'
)
b_analysis2.drop(columns=['ID_x','ID_y'], inplace=True)

In [256]:
# Create summary tables of portfolios
brokerage_df = create_summary_dataframe_with_weighted_date(b_analysis2)
ira_df = create_summary_dataframe_with_weighted_date(r_analysis2)

In [257]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(summary_df, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.95):
    """
    Builds a portfolio analysis table using financial data from Alpha Vantage API,
    with ticker symbols extracted from the provided summary DataFrame.

    Parameters:
    summary_df (pd.DataFrame): Summary DataFrame with at least a 'TICKER' column.
    api_key (str): Alpha Vantage API key for fetching financial data.
    margin_of_safety (float): Margin of safety for VWAP buy threshold (default: 0.9).
    vwap_days (int): Number of days for VWAP calculation (default: 126).
    graham_margin (float): Margin for Graham buy threshold (default: 0.95).

    Returns:
    list: List of lists containing analysis data for each ticker:
          [symbol, market_price, buy_threshold, graham_buy_threshold, exit_price,
           pe_ratio, pb_ratio, dividend_yield, decision]
    """
    # Extract unique tickers from summary_df
    ticker_symbols = [{'symbol': ticker, 'is_etf': False} for ticker in summary_df['TICKER'].unique()]
    
    portfolio = []
    etf_list = ['FBTC', 'ITA', 'SCHF', 'SCHE','SCHX', 'AOA']
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = symbol in etf_list
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Close for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin

        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 38
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, market_price, buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])

        # Convert portfolio list to DataFrame with specified column names
        portfolio_df = pd.DataFrame(portfolio, columns=[
            'ticker', 'price', 'entry_low', 'entry_val', 'exit',
            'P/E', 'P/B', 'DivYield', 'rating'
            ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio_df

In [258]:
ira_cash = 48602.19
ira_historical_return = 1.0225

brokerage_cash = 48602.19
brokerage_historical_return = 0.4858

In [288]:
def build_portfolio_df(portfolio_df, portfolio_price_data, 
                       historical_return, desired_total_exposure=0.9, cash_pos=0,
                       run_date=None):
    quant_rankings = list(portfolio_df['QUANT_RATING'])
    quant_rankings.sort(reverse=True)
    portfolio_df_filtered = portfolio_df[(portfolio_df['TOTAL_QUANTITY'] > 0) | (portfolio_df['QUANT_RATING'] > quant_rankings[9])]
    portfolio_df_sorted = portfolio_df_filtered.sort_values(by='QUANT_RATING', ascending=False).reset_index(drop=True)

    if run_date is None:
        run_date = datetime.now().date()
    run_date = pd.to_datetime(run_date)

    # Merge portfolio holdings with price data for each position
    final_portfolio_df = pd.merge(
            portfolio_df_sorted,
            portfolio_price_data, 
            left_on='TICKER',
            right_on='ticker',
            how='left'
        )
    final_portfolio_df.drop(columns=['ticker'], inplace=True) # remove duplicated column

    # Calculate total value of each held position
    final_portfolio_df['VALUE'] = round(final_portfolio_df['TOTAL_QUANTITY'] * final_portfolio_df['price'], ndigits=2)

    # Calculate value of portfolio, portfolio weights, total return and CAGR for each position
    portfolio_total = final_portfolio_df['VALUE'].sum() + cash_pos
    final_portfolio_df['PW%'] = round(final_portfolio_df['VALUE'] / portfolio_total * 100, ndigits=2)
    final_portfolio_df['TOTAL_RETURN'] = final_portfolio_df['VALUE'] - final_portfolio_df['TOTAL_COST']
    final_portfolio_df['ROI'] = round(final_portfolio_df['TOTAL_RETURN'] / final_portfolio_df['TOTAL_COST'] * 100, ndigits=4)

    # Calculate desired target price to optimize return
    portfolio_cagr = 0
    min_return = 1/3
    desired_return = max(portfolio_cagr, min_return)
    days_held = (run_date - final_portfolio_df['WEIGHTED_AVG_PURCHASE_DATE']).dt.days
    years_held = days_held / 360
    final_portfolio_df['TARGET'] = round(final_portfolio_df['AVG_PURCHASE_PRICE'] * (1+desired_return)**years_held, ndigits=2)

    # Calculate quant-based portfolio allocation and position adjustments to meet desired allocation
    desired_position_size = portfolio_total * desired_total_exposure * (1/20)
    final_portfolio_df['DESIRED_POS'] = desired_position_size * final_portfolio_df['QUANT_RATING'] / 5
    '''
    for company in final_portfolio_df:
        if company['QUANT_RATING']<quant_rankings[19]:
            company['DESIRED_POS'] = 0
        else:
            pass
    '''
    final_portfolio_df['POS_ADJUSTMENT'] = round((final_portfolio_df['DESIRED_POS'] - final_portfolio_df['VALUE']) / final_portfolio_df['price'], ndigits=0)

    # Sorta final portfolio table
    final_portfolio_df = final_portfolio_df.sort_values(by='PW%', ascending=False).reset_index(drop=True)

    return final_portfolio_df, round(portfolio_total, ndigits=2)

In [295]:
ira_price_data = build_analysis_table(ira_df, api_key, margin_of_safety=.975, vwap_days=63)

In [305]:
ira_portfolio_df, ira_total = build_portfolio_df(ira_df, ira_price_data, 
                                                 cash_pos=ira_cash, historical_return=ira_historical_return)

In [306]:
ira_total

143220.82

In [307]:
ira_actions = ira_portfolio_df[['TICKER', 'TOTAL_QUANTITY', 'POS_ADJUSTMENT', 
                                'price', 'entry_low', 'entry_val', 'exit', 'TARGET', 'rating']].copy()
ira_actions['ACTION'] = abs(ira_actions['POS_ADJUSTMENT'] / ira_actions['TOTAL_QUANTITY']) > 0.2
ira_buys = ira_actions[(ira_actions['POS_ADJUSTMENT'] > 0) & (ira_actions['ACTION'] == True)]
ira_sells = ira_actions[(ira_actions['POS_ADJUSTMENT'] < 0) & (ira_actions['ACTION'] == True)]
buys_columns = ['TICKER', 'POS_ADJUSTMENT', 'price', 'entry_low', 'entry_val', 'rating']
sells_columns = ['TICKER', 'POS_ADJUSTMENT', 'price', 'exit', 'TARGET', 'rating']
ira_buys_final = ira_buys[buys_columns].copy()
ira_sells_final = ira_sells[sells_columns].copy()

In [308]:

ira_buys_final

,TICKER,POS_ADJUSTMENT,price,entry_low,entry_val,rating
5,EZPW,74.0,15.68,13.39,26.34,Sell
6,LRN,7.0,161.84,137.64,82.72,Sell
7,COMM,118.0,15.58,9.38,2.17,Sell
8,CLS,10.0,183.61,149.40,46.14,Sell
10,UBER,18.0,95.18,85.75,47.31,Sell
13,TMUS,10.0,257.37,228.33,141.62,Sell
14,ALL,13.0,209.35,192.38,249.70,Sell
15,SKYW,22.0,116.69,103.57,146.33,Sell
16,APP,3.0,416.35,358.87,27.86,Sell
17,WFC,16.0,77.26,75.57,101.26,Hold


In [309]:
ira_sells_final

,TICKER,POS_ADJUSTMENT,price,exit,TARGET,rating
0,ARQT,-235.0,16.2,14.75,17.79,Sell
3,WLDN,-12.0,105.5,83.64,37.04,Sell


In [310]:
ira_portfolio_df[['TICKER', 'QUANT_RATING', 'TOTAL_QUANTITY', 
                  'AVG_PURCHASE_PRICE', 'TOTAL_COST',
                  'price', 'VALUE', 'TOTAL_RETURN', 'ROI']]

,TICKER,QUANT_RATING,TOTAL_QUANTITY,AVG_PURCHASE_PRICE,TOTAL_COST,price,VALUE,TOTAL_RETURN,ROI
0,ARQT,3.30,498.0,15.932,7934.2500,16.20,8067.60,133.3500,1.6807
1,ATGE,4.61,52.0,116.372,6051.3625,128.02,6657.04,605.6775,10.0089
2,EAT,4.42,39.0,124.038,4837.4853,154.76,6035.64,1198.1547,24.7681
3,WLDN,3.49,55.0,36.080,1984.3920,105.50,5802.50,3818.1080,192.4069
4,DXPE,4.77,49.0,105.590,5173.8900,117.10,5737.90,564.0100,10.9011
5,EZPW,4.74,316.0,15.460,4885.3600,15.68,4954.88,69.5200,1.4230
6,LRN,4.63,30.0,107.868,3236.0438,161.84,4855.20,1619.1562,50.0351
7,COMM,4.99,295.0,15.500,4572.5000,15.58,4596.10,23.6000,0.5161
8,CLS,4.95,25.0,-3.456,-86.3962,183.61,4590.25,4676.6462,-5413.0230
9,QTWO,3.05,61.0,79.148,4828.0223,75.20,4587.20,-240.8223,-4.9880


In [267]:
time.sleep(300)

In [268]:
b_analysis_df = build_analysis_table(brokerage_df, api_key, margin_of_safety=.99, vwap_days=32)

Exception fetching price data for ALL: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


In [302]:
brokerage_portfolio_df, brok_total = build_portfolio_df(brokerage_df, b_analysis_df, 
                                                        cash_pos=brokerage_cash,
                                                        historical_return=brokerage_historical_return)

In [303]:
brokerage_portfolio_df = brokerage_portfolio_df[(brokerage_portfolio_df['TOTAL_QUANTITY'] > 0) | (brokerage_portfolio_df['price'] < 100)]

In [304]:
brokerage_portfolio_df

,TICKER,ALPHA_PICKED,QUANT_RATING,ASSET_CLASS,SECTOR,WEIGHTED_AVG_PURCHASE_DATE,TOTAL_COST,TOTAL_QUANTITY,AVG_PURCHASE_PRICE,price,...,P/B,DivYield,rating,VALUE,PW%,TOTAL_RETURN,ROI,TARGET,DESIRED_POS,POS_ADJUSTMENT
0,EZPW,2025-04-01,4.74,Micro Cap,Financials,2025-05-24,2208.153384,140.0,15.773,15.69,...,0.982,0.0000,Sell,2196.60,3.18,-11.553384,-0.5232,16.91,2946.119650,48.0
1,PSIX,2025-06-17,4.99,Micro Cap,Industrials,2025-08-14,1985.520000,24.0,82.730,81.47,...,14.790,0.0000,Hold,1955.28,2.83,-30.240000,-1.5230,83.06,3101.505708,14.0
2,FBTC,NaT,NaN,None,None,2025-04-22,1385.790000,16.0,86.612,98.68,...,NaN,NaN,Buy,1578.88,2.29,193.090000,13.9336,95.25,NaN,NaN
3,OKTA,2024-02-15,3.31,Mid Cap,Information Technology,2025-06-18,1692.100000,17.0,99.535,90.33,...,2.436,0.0000,Hold,1535.61,2.22,-156.490000,-9.2483,104.59,2057.311401,6.0
4,ARQT,2025-03-17,3.30,Micro Cap,Health Care,2025-03-24,1327.643700,83.0,15.996,16.20,...,14.440,0.0000,Sell,1344.60,1.95,16.956300,1.2772,18.00,2051.095959,44.0
5,ATGE,2024-07-15,4.61,Small Cap,Consumer Discretionary,2025-05-30,874.575000,9.0,97.175,128.05,...,3.224,0.0000,Sell,1152.45,1.67,277.875000,31.7726,103.67,2865.318900,13.0
6,QTWO,2024-12-02,3.05,Small Cap,Information Technology,2025-08-05,1146.530500,15.0,76.435,75.12,...,8.090,0.0000,Hold,1126.80,1.63,-19.730500,-1.7209,77.29,1895.709901,10.0
7,FRSH,2025-07-29,3.06,Small Cap,Information Technology,2025-08-01,1161.666300,85.0,13.667,13.24,...,3.957,0.0000,Hold,1125.40,1.63,-36.266300,-3.1219,13.87,1901.925344,59.0
8,PYPL,2025-01-15,3.39,Large Cap,Financials,2025-06-25,1092.669800,16.0,68.292,69.06,...,3.288,0.0000,Hold,1104.96,1.60,12.290200,1.1248,71.36,2107.034940,15.0
9,LX,2025-07-29,4.40,Micro Cap,Information Technology,2025-08-03,1052.029000,161.0,6.534,6.24,...,0.647,0.0454,Hold,1004.64,1.45,-47.389000,-4.5045,6.62,2734.794612,277.0


In [272]:
'''def setup_driver():
    chrome_options = Options()
    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")  # Connect to existing Chrome
    driver = webdriver.Chrome(options=chrome_options)  # No need for user-data-dir here
    return driver

def scrape_quant_ratings(symbol, driver, days=63):
    url = f'https://seekingalpha.com/symbol/{symbol}/ratings/quant-ratings'
    driver.get(url)
    time.sleep(5)  # Extra delay for dynamic content
    
    try:
        # Wait for table to load
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
    except Exception as e:
        driver.save_screenshot(f'error_{symbol}.png')
        with open(f'page_source_{symbol}.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
        print(f"Page load failed for {symbol}: {e}. Saved screenshot and page source for debugging.")
        raise Exception(f"Failed to load page for {symbol}")
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Find the quant rating history table
    table = soup.find('table')
    
    if not table:
        raise Exception("Table not found")
    
    rows = table.find_all('tr')[1:]  # Skip header
    data = []
    for row in rows:
        cols = row.find_all('td')
        if len(cols) >= 4:
            date_str = cols[0].text.strip()
            try:
                date = datetime.strptime(date_str, '%m/%d/%Y')
            except ValueError:
                continue
            quant_score_str = cols[3].text.strip()
            try:
                quant_score = float(quant_score_str)
            except ValueError:
                quant_score = np.nan
            data.append({'date': date, 'quant_score': quant_score})
    
    df = pd.DataFrame(data)
    df = df.sort_values('date', ascending=False)
    
    # Filter last 63 days (current date as 02:24 PM MDT, Aug 10, 2025)
    end_date = datetime(2025, 8, 10, 14, 24)  # 2:24 PM MDT
    start_date = end_date - timedelta(days=days)
    df_filtered = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    
    return df_filtered

def compute_stats(df):
    scores = df['quant_score'].dropna()
    if len(scores) == 0:
        return {'average': np.nan, 'median': np.nan}
    return {
        'average': scores.mean(),
        'median': scores.median()
    }

# Main function
def main(companies):
    driver = setup_driver()
    try:
        results = {}
        for symbol in companies:
            try:
                df = scrape_quant_ratings(symbol, driver)
                stats = compute_stats(df)
                results[symbol] = stats
            except Exception as e:
                print(f"Error for {symbol}: {e}")
                results[symbol] = {'average': np.nan, 'median': np.nan}
        
        # Output results
        results_df = pd.DataFrame.from_dict(results, orient='index')
        print(results_df)
    finally:
        driver.quit()

# Example usage
companies = ['DXPE', 'AAPL', 'GOOG']  # Replace with your list
main(companies)
'''

'def setup_driver():\n    chrome_options = Options()\n    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")  # Connect to existing Chrome\n    driver = webdriver.Chrome(options=chrome_options)  # No need for user-data-dir here\n    return driver\n\ndef scrape_quant_ratings(symbol, driver, days=63):\n    url = f\'https://seekingalpha.com/symbol/{symbol}/ratings/quant-ratings\'\n    driver.get(url)\n    time.sleep(5)  # Extra delay for dynamic content\n    \n    try:\n        # Wait for table to load\n        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, \'table\')))\n    except Exception as e:\n        driver.save_screenshot(f\'error_{symbol}.png\')\n        with open(f\'page_source_{symbol}.html\', \'w\', encoding=\'utf-8\') as f:\n            f.write(driver.page_source)\n        print(f"Page load failed for {symbol}: {e}. Saved screenshot and page source for debugging.")\n        raise Exception(f"Failed to load page for {symbol

In [273]:
# Machine Learning

In [274]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [275]:
# Feature selection and normalization

In [276]:
# Build model

In [277]:
# Train model

In [278]:
# Evaluate model

In [279]:
# Refine model through hyperparameter tuning